# Intro to Machine Learning 2

## Classification of phases of matter

*Machine Learning Phases of Matter*, Juan Carrasquilla and Roger Melko, Nature Physics 13, 431 (2017)

(https://arxiv.org/abs/1605.01735)

#### Utilities

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tarfile

plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
def extract_images(filename,L):
    """Extract the images into a 4D uint8 numpy array [index, y, x, depth]."""
    print ('Extracting', filename)
    data=np.loadtxt(filename,dtype='uint8')
    dim=data.shape[0]
    data=data.reshape(dim,L,L,1) 
    print(data.shape)
    return data

def extract_labels(nlabels,filename):
    """Extract the labels into a 1D uint8 numpy array [index]."""
    print('Extracting', filename)

    labels=np.loadtxt(filename,dtype='uint8')
    print("LABELS")
    print(labels.shape)
    return labels

class DataSet(object):
    def __init__(self, images, labels, fake_data=False):
        if fake_data:
            self._num_examples = 10000
        else:
            assert images.shape[0] == labels.shape[0], (
                "images.shape: %s labels.shape: %s" % (images.shape,
                                                       labels.shape))
            self._num_examples = images.shape[0]
            # Convert shape from [num examples, rows, columns, depth]
            # to [num examples, rows*columns] (assuming depth == 1)
            assert images.shape[3] == 1
            images = images.reshape(images.shape[0],
                                    images.shape[1] * images.shape[2])
            # Convert from [0, 255] -> [0.0, 1.0].
            images = images.astype(np.float32)
            # images = numpy.multiply(images, 1.0 / 255.0) # commented since it is ising variables
            images = np.multiply(images, 1.0 ) # multiply by one, instead
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, fake_data=False):
        """Return the next `batch_size` examples from this data set."""
        if fake_data:
            fake_image = [1.0 for _ in xrange(400)]
            fake_label = 0
            return [fake_image for _ in xrange(batch_size)], [
                fake_label for _ in xrange(batch_size)]
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._images = self._images[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._images[start:end], self._labels[start:end]

def read_data_sets(nlabels,L):
    class DataSets(object):
        pass
    data_sets = DataSets()
    TRAIN_IMAGES = 'Xtrain.txt'
    TRAIN_LABELS = 'ytrain.txt'
    TEST_IMAGES = 'Xtest.txt'
    TEST_LABELS = 'ytest.txt'
    VALIDATION_SIZE = 0
    train_images = extract_images(TRAIN_IMAGES,L)
    train_labels = extract_labels(nlabels,TRAIN_LABELS)
    test_images = extract_images(TEST_IMAGES,L)
    test_labels = extract_labels(nlabels,TEST_LABELS)
    data_sets.train = DataSet(train_images, train_labels)
    data_sets.test = DataSet(test_images, test_labels)
    return data_sets

def evalute_model(x,y,y_model):
    acc = np.zeros((Ntemp))
    output = np.zeros((Ntemp,2))

    # Average output of neural net over the test set
    ii=0
    for i in range(Ntemp):
        av=0.0
        for j in range(samples_per_T):
            test_image = data.test.images[ii,:].reshape((1,L*L))
            test_label = np.asarray([1],data.test.labels[ii])
            batch_valid=(test_image,test_label)  
            res=sess.run(y_model,feed_dict={x: batch_valid[0], y: batch_valid[1]})
            av=av+res 
            ii=ii+1 
        av=av/samples_per_T
        output[i,0] = av[0,0]
        output[i,1] = av[0,1]

    #Average accuracy vs temperature over the test set
    for ii in range(Ntemp):
        test_images = data.test.images[ii*samples_per_T:ii*samples_per_T+samples_per_T,:].reshape(samples_per_T,L*L)
        test_labels = np.asarray(data.test.labels[ii*samples_per_T:ii*samples_per_T+samples_per_T])
        batch_valid = (test_images,test_labels)
        test_accuracy = sess.run(accuracy,feed_dict={
            x:batch_valid[0], y: batch_valid[1]}) 
        acc[ii] = test_accuracy

    return acc,output

### Load Data

In [ ]:
# temperature list at which the training/test sets were generated
tlist=[1.0000000000000000,1.0634592657106510,1.1269185314213019,1.1903777971319529,1.2538370628426039,1.3172963285532548,1.3807555942639058,1.4442148599745568,1.5076741256852078,1.5711333913958587,1.6345926571065097,1.6980519228171607,1.7615111885278116,1.8249704542384626,1.8884297199491136,1.9518889856597645,2.0153482513704155,2.0788075170810667,2.1422667827917179,2.2057260485023691,2.3326445799236715,2.3961038456343227,2.4595631113449739,2.5230223770556250,2.5864816427662762,2.6499409084769274,2.7134001741875786,2.7768594398982298,2.8403187056088810,2.9037779713195322,2.9672372370301834,3.0306965027408346,3.0941557684514858,3.1576150341621370,3.2210742998727881,3.2845335655834393,3.3479928312940905,3.4114520970047417,3.4749113627153929,3.5383706284260401]

# Description of the input data 
Ntemp=40           # number of different temperatures used in the training and testing data
samples_per_T=250  # number of samples per temperature value in the testing set
Nord=20            # number of temperatures in the ordered phase

# Model Parameters
L=30   # linear size of the lattice
K=2    # number of phases
N=L*L  # number of spins

# Loading the data
tar = tarfile.open('CompPhys2-machinelearning2_DATA.tar.gz', "r:gz")
tar.extractall()
tar.close() 
data = read_data_sets(K,L)

### Neural Network

#### Network parameters

In [ ]:
# Model Parameters
L2 = 0.0                  # regularization parameter
batch_size = 50           # batch size
epochs = 4000             # training steps
nh = 1                    # number of hidden neurons
learning_rate = 0.1       # learning rate

#### Variables

In [ ]:
# Variables
W = tf.Variable(tf.truncated_normal([ N, nh], mean=0.0,stddev=0.1, dtype=tf.float32))
W1 = tf.Variable(tf.truncated_normal([ nh, K], mean=0.0,stddev=0.1, dtype=tf.float32))
b = tf.Variable(0.1*tf.ones([nh]), dtype=tf.float32)
b1 = tf.Variable(0.1*tf.ones([K]), dtype=tf.float32)

# Placeholders
x = tf.placeholder(tf.float32, [None, N])
y = tf.placeholder(tf.int32,[None])

#### Model

In [ ]:
#############################
# Define here the forward pass
# Use sigmoid neurons also in the last layer (we only have two classes here)

def forward_pass(x,w,b):
    #.....
    
# This is the output of the model
y_model= #......


# Cost function     
cost = #.....         +(L2)*(tf.nn.l2_loss(W)+tf.nn.l2_loss(W1))

# Optimizer
optimizer= tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cost)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_model,1), tf.argmax(tf.one_hot(y,depth=K),1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#### Training

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

W_init=sess.run(W)
b_init=sess.run(b)

for i in range(epochs+1):
    batch = data.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

    if i%250 == 0:
        train_accuracy = sess.run(accuracy,feed_dict={
            x:batch[0], y: batch[1]})
        test_accuracy = sess.run(accuracy, feed_dict={
            x: data.test.images, y: data.test.labels})
        loss = sess.run(cost,feed_dict={x: batch[0], y: batch[1]})
        print("Epoch = %d \t loss %f \t Trainset Accuracy = %.4f \t Testset Accuracy = %.4f" % (i, loss,train_accuracy,test_accuracy))
        W_opt=sess.run(W)
        b_opt=sess.run(b)
        acc,output=evalute_model(x,y,y_model)

#### Plots

In [ ]:
plt.rcParams['figure.figsize'] = (9.0, 9.0)
plt.subplot(211)
c = plt.cm.spectral((1)/4.,1)
plt.plot(tlist,output[:,0],marker='D',markersize=5,color=c,linewidth=1,label='FM neuron')
c = plt.cm.spectral((2)/4.,1)
plt.plot(tlist,output[:,1],marker='s',markersize=5,color=c,linewidth=1,label='PM neuron')
leg = plt.legend(loc='right',numpoints=1,markerscale=1.0,fontsize=15,labelspacing=0.1)
plt.ylabel('Average output layer', fontsize=15)
plt.xlabel(r'$T$', fontsize=15,labelpad=0)
plt.xlim([1,3.5383706284260401])
plt.ylim([0,1])
x_tc=[2.26918,2.26918]
y_tc=[0,1]
plt.plot(x_tc, y_tc,color='#FFA500',linewidth=2)


plt.subplot(212)
c = 'r'
plt.plot(tlist,acc,marker='*',markersize=5,color=c,linewidth=1)
plt.ylabel('Average accuracy', fontsize=15)
plt.xlabel(r'$T$', fontsize=15,labelpad=0)
plt.xlim([1,3.5383706284260401])
plt.ylim([0.6,1])


x_tc=[2.26918,2.26918]
y_tc=[0.6,1]
plt.plot(x_tc, y_tc,color='#FFA500',linewidth=2)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (6.0, 6.0)
magnetizations = sess.run(2*tf.reduce_mean(data.test.images,1)-1)
activations = sess.run(tf.matmul(x,W_opt)+b_opt,feed_dict={x:data.test.images})

c = plt.cm.spectral((1)/4.,1)
plt.plot(magnetizations,activations[:,0],marker='D',markersize=0.5,linewidth=0.0,color=c)
c = plt.cm.spectral((2)/4.,1)
plt.plot(magnetizations,activations[:,1],marker='D',markersize=0.5,linewidth=0.0,color=c)
c = 'k'
plt.plot(magnetizations,activations[:,2],marker='D',markersize=0.5,linewidth=0.0,color=c)
plt.ylabel('$Wx+b$', fontsize=15)
plt.xlabel(r'$m(x)$', fontsize=15,labelpad=0)
plt.xlim([-1,1])
#plt.ylim([0,1])
x_grid=[0,0]
y_grid=[np.min(activations),np.max(activations)]
plt.plot(x_grid, y_grid,color='r',linewidth=1)
y_grid=[0,0]
x_grid=[-1,1]
plt.plot(x_grid, y_grid,color='r',linewidth=1)
plt.show()